In [1]:
# Imports & Paths

import os
import numpy as np
import pandas as pd
import nibabel as nib
import SimpleITK as sitk
import random
from tqdm import tqdm


In [2]:


def extract_vessel_names_from_files(folder):
    # List all files in the Lumen folder
    files = os.listdir(folder)
    
    # Extract vessel names from the filenames
    vessel_names = {}
    vessel_labels = {}
    vessel_map = {}
    for file in files:
        if file.endswith("Centerline.txt"):
            # Extract vessel name from the filename (e.g., vessel_0_LAD_Centerline.txt -> LAD)
            parts= file.split('_')
            vessel_name = parts[2]
            vessel_label = int(parts[1])
            vessel_names[vessel_label] = vessel_name
            vessel_labels[vessel_name] = vessel_label  # Optional: reverse mapping
            vessel_map[vessel_name] = f"{vessel_label}_{vessel_name}"
    
    return vessel_names, vessel_labels, vessel_map


In [3]:
# Global Maps

LABEL_MAP = {
    1: "myocardium of LV (Myo)",
    2: "left atrium (LA)",
    3: "left ventricle (LV)",
    4: "right atrium (RA)",
    5: "right ventricle (RV)",
    6: "ascending aorta (AO)",
    7: "pulmonary artery (PA)",
    8: "descending aorta (DO)"
}

MIN_SIZE = (40, 40, 20)
MAX_SIZE = (180, 180, 120)


In [4]:
# Utility Functions (NIfTI & MHD)

def load_nifti(path):
    nii = nib.load(path)
    return nii.get_fdata(), nii.affine

def load_mhd(path):
    img = sitk.ReadImage(path)
    arr = sitk.GetArrayFromImage(img)
    return arr, img.GetSpacing(), img.GetOrigin(), img.GetDirection()

def save_ct_as_nifti(cube_ct, spacing, output_path):
    affine = np.array([
        [spacing[2], 0,          0,          0],
        [0,          spacing[1], 0,          0],
        [0,          0,          spacing[0], 0],
        [0,          0,          0,          1]
    ])
    nib.save(nib.Nifti1Image(cube_ct.astype(np.float32), affine), output_path)

def save_seg_as_nifti(cube, affine, output_path):
    nib.save(nib.Nifti1Image(cube.astype(np.uint8), affine), output_path)

from scipy.ndimage import zoom

def process_nifti(data, output_path, target_shape=(32, 256, 256)):
    """
    Reads a NIfTI file, resizes it, normalizes it, and saves as .npy
    
    Args:
        nifti_path: Path to input .nii.gz
        output_path: Path to save .npy
        target_shape: Tuple (Depth, Height, Width) -> Default (32, 256, 256)
    """
    # 1. Load the NIfTI file
    #img = nib.load(nifti_path)
    #data = img.get_fdata()

    # 2. Check and Fix Orientation
    # Standard NIfTI loads as (Height, Width, Depth) or (X, Y, Z).
    # We want (Depth, Height, Width).
    # usually this means moving the last axis (Z) to the front.
    data = np.transpose(data, (2, 0, 1)) # (D, H, W)

    # 3. Resize (Resample) the Volume
    # We calculate how much we need to zoom/shrink each axis to hit the target
    current_depth, current_height, current_width = data.shape
    target_depth, target_height, target_width = target_shape

    depth_factor = target_depth / current_depth
    height_factor = target_height / current_height
    width_factor = target_width / current_width

    # Perform cubic interpolation (order=3) to resize
    # This might take a few seconds per image
    data_resized = zoom(data, (depth_factor, height_factor, width_factor), order=1) 

    # 4. Normalize to [0, 1]
    # For CT scans, it is often good to clip huge values first (e.g., bone/air)
    # Simple Min-Max normalization:
    min_val = np.min(data_resized)
    max_val = np.max(data_resized)
    
    if max_val - min_val != 0:
        data_norm = (data_resized - min_val) / (max_val - min_val)
    else:
        data_norm = np.zeros(data_resized.shape)

    # 5. Add Channel Dimension
    # Target requires: (Channels, Depth, Height, Width) -> (1, 32, 256, 256)
    data_final = np.expand_dims(data_norm, axis=0)

    # 6. Verify and Save
    print(f"Original shape: {data.shape}")
    print(f"Final shape:    {data_final.shape}")
    print(f"Value range:    {np.min(data_final):.2f} - {np.max(data_final):.2f}")
    
    # Ensure data type is float32 (standard for PyTorch)
    data_final = data_final.astype(np.float32)
    
    np.save(output_path, data_final)
    print(f"Saved to: {output_path}")


In [5]:
# Cube Sampling

def sample_cube(shape):
    D, H, W = shape

    dz = np.random.randint(MIN_SIZE[0], MAX_SIZE[0])
    dy = np.random.randint(MIN_SIZE[1], MAX_SIZE[1])
    dx = np.random.randint(MIN_SIZE[2], MAX_SIZE[2])

    z1 = random.randint(0, D - dz - 1)
    y1 = random.randint(0, H - dy - 1)
    x1 = random.randint(0, W - dx - 1)

    return (z1, z1 + dz, y1, y1 + dy, x1, x1 + dx)


In [6]:
# Task 1 (Anatomy)

def get_labels(seg_vol, cube):
    z1, z2, y1, y2, x1, x2 = cube
    patch = seg_vol[z1:z2, y1:y2, x1:x2]
    labels = np.unique(patch)
    return [int(l) for l in labels if l != 0]

def generate_prompt(labels):
    if not labels:
        return "This cube contains no major anatomical structures."
    names = [LABEL_MAP[l] for l in labels]
    return "This cube contains " + ", ".join(names) + "."


In [7]:
# Task 2 (Detect vessels using mask + centerline)

# Detect vessels from vessel_seg.mhd mask
def detect_vessels_from_seg(vessel_seg, cube):
    z1,z2,y1,y2,x1,x2 = cube
    mask = vessel_seg[z1:z2, y1:y2, x1:x2]
    vessels = np.unique(mask)
    vessels = [v for v in vessels if v in VESSEL_NAME_MAP]
    return [VESSEL_NAME_MAP[v] for v in vessels]

# (Optional) centerline detection from Lumen folder
def convert_world_to_voxel_zyx(pts_lps):
    x_ras = -pts_lps[:,0]
    y_ras = -pts_lps[:,1]
    z_ras = pts_lps[:,2]

    i = (69.7827 - x_ras) / 0.434
    j = (132.9   - y_ras) / 0.434
    k = (1904.5  - z_ras) / 0.25

    return np.vstack([k,j,i]).T

def load_vessel_world(folder, values_list):
    vessel_world = {}
    for name in values_list:
        vessel_world[name] = np.loadtxt(f"{folder}/vessel_{name}_Centerline.txt")
    return vessel_world

def load_and_convert_vessels(folder, values_list):
    """Load world coordinates → convert to voxel coords."""
    world_dict = load_vessel_world(folder, values_list)
    return {name: convert_world_to_voxel_zyx(pts)
            for name, pts in world_dict.items()}

def find_vessels_in_cube(vessel_voxel_zyx, cube):
    """Return vessel names + rough location inside cube."""
    z1, z2, y1, y2, x1, x2 = cube
    results = []

    for name, pts in vessel_voxel_zyx.items():
        inside = [
            (vz, vy, vx)
            for (vz, vy, vx) in pts
            if (z1 <= vz < z2 and y1 <= vy < y2 and x1 <= vx < x2)
        ]
        if not inside:
            continue

        inside = np.array(inside)
        mz, my, mx = np.mean(inside, axis=0)

        # center thresholds
        z_mid = (z1 + z2) / 2
        y_mid = (y1 + y2) / 2
        x_mid = (x1 + x2) / 2

        # Rough region description
        desc_z = "upper" if mz < z_mid else "lower"
        desc_y = "anterior" if my < y_mid else "posterior"
        desc_x = "left" if mx < x_mid else "right"

        loc = f"{desc_z}-{desc_y}-{desc_x} region"
        split_name = name.split('_')[1]
        results.append({"vessel": split_name, "location": loc})

    return results



In [8]:
# Task 3 (Stenosis)

def cumulative_distance(pts):
    diffs = np.diff(pts, axis=0)
    seg = np.linalg.norm(diffs, axis=1)
    return np.concatenate([[0], np.cumsum(seg)])

def extract_stenosis_world(pts, start, end):
    cum = cumulative_distance(pts)
    mask = (cum >= start) & (cum <= end)
    return pts[mask]

def stenosis_in_cube(pts_vox, cube):
    z1,z2,y1,y2,x1,x2 = cube
    inside = [
        (vz,vy,vx)
        for (vz,vy,vx) in pts_vox
        if z1<=vz<z2 and y1<=vy<y2 and x1<=vx<x2
    ]
    return inside


def process_stenosis_for_cube(cube, stenosis_df, STENOSIS_FOLDER, vessel_map):
    results = []
    prompt_parts = []

    for _, row in stenosis_df.iterrows():
        vessel = row["Lesion vessel"]
        s_start = row["Plaque start location in centreline, mm"]
        s_end   = row["Plaque end location in centreline, mm"]
        grade   = row["Stenosis grade"]

        stenosis_name = vessel_map[vessel]
        path = f"{STENOSIS_FOLDER}/vessel_{stenosis_name}_Centerline.txt"

        pts = np.loadtxt(path)
        sten_world = extract_stenosis_world(pts, s_start, s_end)
        if len(sten_world) == 0:
            continue

        sten_vox = convert_world_to_voxel_zyx(sten_world)
        inside = stenosis_in_cube(sten_vox, cube)

        if inside:
            results.append({
                "vessel": vessel,
                "grade": grade,
                "count_points_inside": len(inside)
            })
            prompt_parts.append(f"{vessel} stenosis (grade {grade})")

    if prompt_parts:
        return results, "This cube contains " + ", ".join(prompt_parts) + "."
    return [], "This cube contains no stenosis."


In [9]:
# Main Dataset Generator
import dicom2nifti
import glob

def generate_dataset(CTCA_FOLDER, OUTPUT_DIR, PDCase, target_valid=2):
    

    dicom_directory = os.path.join(CTCA_FOLDER, "DICOM")

    dicom2nifti.convert_directory(dicom_directory, OUTPUT_DIR)

    nii_files = glob.glob(os.path.join(OUTPUT_DIR, "*.nii.gz"))
    
    if nii_files:
        CT_PATH = nii_files[0]  # Take the first .nii.gz file found
        print(f"Found CT file: {CT_PATH}")
    else:
        print("No .nii.gz file found in the output directory")
        print(f"CT file missing, skipping dataset generation for {PDCase}.")
        CT_PATH = None
        return None

        
    SEG_PATH = os.path.join(CTCA_FOLDER, f"{PDCase}_CT_WHS.nii.gz")
    VESSEL_SEG_MHD = os.path.join(CTCA_FOLDER, f"{PDCase}_seg.mhd")
    LUMEN_FOLDER = os.path.join(CTCA_FOLDER, "Lumen")
    STENOSIS_FOLDER = os.path.join(CTCA_FOLDER, "Stenosis")
    LABEL_XLS = os.path.join(CTCA_FOLDER, "Label/Labels.xlsx")

    df = pd.read_excel(LABEL_XLS)
    df = df.drop(df.columns[0], axis=1)      # drop "case"
    df[df.columns[0]] = df[df.columns[0]].ffill()
    



    # List all files in the Lumen folder
    files = os.listdir(LUMEN_FOLDER)
    
    # Extract vessel names from the filenames
    vessel_names = {}
    vessel_labels = {}
    vessel_map = {}
    for file in files:
        if file.endswith("Centerline.txt"):
            # Extract vessel name from the filename (e.g., vessel_0_LAD_Centerline.txt -> LAD)
            parts= file.split('_')
            vessel_name = parts[2]
            vessel_label = int(parts[1])
            vessel_names[vessel_label] = vessel_name
            vessel_labels[vessel_name] = vessel_label  # Optional: reverse mapping
            vessel_map[vessel_name] = f"{vessel_label}_{vessel_name}"


    
    ct, ct_affine = load_nifti(CT_PATH)
    seg, seg_affine = load_nifti(SEG_PATH)
    vessel_seg, spacing, origin, direction = load_mhd(VESSEL_SEG_MHD)

    os.makedirs(OUTPUT_DIR, exist_ok=True)

    valid = 0
    total = 0

    pbar = tqdm(total=target_valid)

    while valid < target_valid:
        total += 1

        # Sample cube
        cube = sample_cube(ct.shape)
        z1, z2, y1, y2, x1, x2 = cube

        cube_ct = ct[z1:z2, y1:y2, x1:x2]
        cube_seg = seg[z1:z2, y1:y2, x1:x2]
        cube_vseg = vessel_seg[z1:z2, y1:y2, x1:x2]

        # Task 1 — Anatomical labels
        labels = get_labels(seg, cube)
        anatomy_prompt = generate_prompt(labels)

        # Task 2 — Vessel detection
        #vessels_present = detect_vessels_from_seg(vessel_seg, cube)
        values_list = list(vessel_map.values())
        vessel_vox = load_and_convert_vessels(LUMEN_FOLDER, values_list)
        vessels_present_by_centerline = find_vessels_in_cube(vessel_vox, cube)


        # Task 3 — Stenosis detection
        stenosis_results, stenosis_prompt = process_stenosis_for_cube(cube, df, STENOSIS_FOLDER, vessel_map)

        if stenosis_results:
            valid += 1
            print(total)
            pbar.update(1)

        # Create cube directory and save files
        cube_dir = f"{OUTPUT_DIR}/cube_{total:05d}"
        os.makedirs(cube_dir, exist_ok=True)

        save_ct_as_nifti(cube_ct, spacing=(0.25, 0.434, 0.434), output_path=f"{cube_dir}/ct_cube.nii.gz")
        process_nifti(cube_ct, output_path=f"{cube_dir}/correct_ct_cube.npy", target_shape=(32, 256, 256))

        save_seg_as_nifti(cube_seg, ct_affine, f"{cube_dir}/seg_cube.nii.gz")
        save_seg_as_nifti(cube_vseg, ct_affine, f"{cube_dir}/vessel_seg_cube.nii.gz")

        np.save(f"{cube_dir}/ct.npy", cube_ct)
        np.save(f"{cube_dir}/seg.npy", cube_seg)
        np.save(f"{cube_dir}/vessel_seg.npy", cube_vseg)

        # Format output into nice sections for meta.txt
        meta_output = f"Cube Index: {total}\n"
        meta_output += f"Coordinates: {cube}\n\n"

        # Task 1 - Anatomical Labels
        meta_output += "Task 1: Anatomical Labels\n"
        meta_output += "---------------------------\n"
        meta_output += f"Labels found: {labels}\n"
        meta_output += "\n".join([f"- {LABEL_MAP.get(l, 'Unknown label')}" for l in labels]) + "\n\n"
        meta_output += f"Prompt: {anatomy_prompt}\n\n"

        # Task 2 - Vessel Detection
        meta_output += "Task 2: Vessel Detection (Coronary Arteries)\n"
        meta_output += "-------------------------\n"
        #meta_output += f"Detected Vessels(done by mhd):\n"
        #for vessel in vessels_present:
        #    meta_output += f"- {vessel} (location: TBD)\n"  # Add location based on actual logic
        meta_output += f"Detected Vessels(done by Lumen folder centerline filter):\n"
        meta_output += f"{vessels_present_by_centerline}\n"
        meta_output += "\n"

        # Task 3 - Stenosis Detection
        meta_output += "Task 3: Stenosis Detection\n"
        meta_output += "----------------------------\n"
        if stenosis_results:
            meta_output += f"Stenosis Results:\n"
            for result in stenosis_results:
                meta_output += f"- Vessel: {result['vessel']}, Grade: {result['grade']}, Points Inside: {result['count_points_inside']}\n"
            meta_output += f"\nStenosis Prompt: {stenosis_prompt}\n"
        else:
            meta_output += "No stenosis detected in this cube.\n"

        # Save metadata in the final format
        with open(f"{cube_dir}/meta.txt", "w") as f:
            f.write(meta_output)

    pbar.close()
    print("DONE:", valid, "valid cubes generated.")


In [10]:
import os

def findPath_and_executeEachCases_folder(data_folder, output_folder_path):
    COUNT = 0
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)
        print(f"Created output folder: {output_folder_path}")

    # List all case folders inside the "data" folder
    case_folders = [f for f in os.listdir(data_folder) if os.path.isdir(os.path.join(data_folder, f))]
    
    # Iterate through each case folder
    for case_folder in case_folders:
        case_folder_path = os.path.join(data_folder, case_folder)
        
        # List all subfolders in the case folder (subfolder should be only one, named with date)
        subfolders = [f for f in os.listdir(case_folder_path) if os.path.isdir(os.path.join(case_folder_path, f))]
        if len(subfolders) > 1:
            print(f"More then one subfolder(folder named with date inside Case Folder PD{case_folder})")
            continue

        # Look for the "CTCA" folder inside each subfolder
        for subfolder in subfolders:
            subfolder_path = os.path.join(case_folder_path, subfolder)
            
            # Check if the "CTCA" folder exists inside this subfolder
            ctca_folder_path = os.path.join(subfolder_path, "CTCA")
            if os.path.exists(ctca_folder_path):
                case_output_folder = os.path.join(output_folder_path, f"Case_{case_folder}")
                os.makedirs(case_output_folder, exist_ok=True)
                print(f"Created output folder for case {case_folder}: {case_output_folder}")
                
                # Run the dataset generation for each case
                generate_dataset(ctca_folder_path, case_output_folder, case_folder)  # Pass output folder for each case
                print(f"Dataset generated for case {case_folder} at {case_output_folder}")        
                COUNT += 1        

    print(f"Finshed {COUNT} cases.")
    return None 

# Example usage
data_folder = "/home/jiacheng/Documents/nas/data"  # Path to your "data" folder
output_folder_path = "/home/jiacheng/Documents/nas/data_prep_output/folder_forwholedataset"
findPath_and_executeEachCases_folder(data_folder, output_folder_path)



Created output folder for case PD489: /home/jiacheng/Documents/nas/data_prep_output/folder_forwholedataset/Case_PD489


Found CT file: /home/jiacheng/Documents/nas/data_prep_output/folder_forwholedataset/Case_PD489/5_half_688ms_077scectahalf688msfc42.nii.gz


  0%|          | 0/2 [00:00<?, ?it/s]

Original shape: (78, 127, 67)
Final shape:    (1, 32, 256, 256)
Value range:    0.00 - 1.00
Saved to: /home/jiacheng/Documents/nas/data_prep_output/folder_forwholedataset/Case_PD489/cube_00001/correct_ct_cube.npy
Original shape: (63, 87, 71)
Final shape:    (1, 32, 256, 256)
Value range:    0.00 - 1.00
Saved to: /home/jiacheng/Documents/nas/data_prep_output/folder_forwholedataset/Case_PD489/cube_00002/correct_ct_cube.npy
Original shape: (43, 91, 108)
Final shape:    (1, 32, 256, 256)
Value range:    0.00 - 1.00
Saved to: /home/jiacheng/Documents/nas/data_prep_output/folder_forwholedataset/Case_PD489/cube_00003/correct_ct_cube.npy
Original shape: (86, 176, 119)
Final shape:    (1, 32, 256, 256)
Value range:    0.00 - 1.00
Saved to: /home/jiacheng/Documents/nas/data_prep_output/folder_forwholedataset/Case_PD489/cube_00004/correct_ct_cube.npy
Original shape: (44, 80, 130)
Final shape:    (1, 32, 256, 256)
Value range:    0.00 - 1.00
Saved to: /home/jiacheng/Documents/nas/data_prep_output/

 50%|█████     | 1/2 [00:27<00:27, 27.97s/it]

...

47
Original shape: (117, 142, 128)
Final shape:    (1, 32, 256, 256)
Value range:    0.00 - 1.00
Saved to: /home/jiacheng/Documents/nas/data_prep_output/folder_forwholedataset/Case_PD521/cube_00047/correct_ct_cube.npy
Original shape: (102, 93, 78)
Final shape:    (1, 32, 256, 256)
Value range:    0.00 - 1.00
Saved to: /home/jiacheng/Documents/nas/data_prep_output/folder_forwholedataset/Case_PD521/cube_00048/correct_ct_cube.npy


100%|██████████| 2/2 [01:05<00:00, 27.70s/it]

49
Original shape: (82, 65, 159)
Final shape:    (1, 32, 256, 256)
Value range:    0.00 - 1.00
Saved to: /home/jiacheng/Documents/nas/data_prep_output/folder_forwholedataset/Case_PD521/cube_00049/correct_ct_cube.npy


100%|██████████| 2/2 [01:07<00:00, 33.56s/it]

DONE: 2 valid cubes generated.
Dataset generated for case PD521 at /home/jiacheng/Documents/nas/data_prep_output/folder_forwholedataset/Case_PD521
Finshed 2 cases.
